![Banner](images/banner.png)

# Queries - SELECT and WITH Statements

In [ ]:
import cx_Oracle
import platform
import os

if platform.system() == 'Darwin':
    cx_Oracle.init_oracle_client(lib_dir=os.environ.get("HOME")+"/instantclient_19_8")
elif platform.system() == 'Windows':
     cx_Oracle.init_oracle_client(lib_dir=r"C:\oracle\instantclient_19_14")

In [ ]:
un       = "pythondemo"
pw       = "welcome"

adminun  = "system"
adminpw  = "oracle"

cs       = "localhost/orclpdb1"

connection = cx_Oracle.connect(user=un, password=pw, dsn=cs)

systemconnection = cx_Oracle.connect(user=adminun, password=adminpw, dsn=cs)

## A key tuning goal is to reduce "round-trips"


> "A server round-trip is defined as the trip from the client to the server and back to the client."

Round-trips affect performance and system scalability.

<!-- Make every round-trip useful

- don't select more data than needed
- fetch multiple rows at a time
- insert multiple rows at a time
- don't overuse commit or rollback -->

![Round-trip](images/roundtrip.png)

# Fetching Rows

Documentation reference link: [Fetch Methods](https://cx-oracle.readthedocs.io/en/latest/user_guide/sql_execution.html#fetch-methods)

In [ ]:
# Fetching single rows ("fetchone()")

with connection.cursor() as cursor:
    
    cursor.execute("select * from SampleQueryTab where id = 6")
    
    row = cursor.fetchone()
    print(row)
    

In [ ]:
# Fetching all rows from a small table of not too big size ("fetchall()")

with connection.cursor() as cursor:
       
    cursor.execute("select * from SampleQueryTab")
    
    rows = cursor.fetchall()
    
    for r in rows:
        print(r)
      

In [ ]:
# Fetching all rows from a big table ("fetchmany()")

with connection.cursor() as cursor:
 
    cursor.execute("select * from SampleQueryTab")  # pretend this is big
    
    while True:
        rows = cursor.fetchmany()  # get a batch rows (of size cursor.arraysize=100)
        for r in rows:
            print(r)
        if len(rows) < cursor.arraysize:
            break


## Round-trip comparison - prefetchrows & arraysize

Documentation reference link: [Tuning cx_Oracle](https://cx-oracle.readthedocs.io/en/latest/user_guide/tuning.html)

Regardless of which fetch method is used, there are two tuning parameters that affect internal buffering of fetched rows.  These parameters do not affect how, or when, rows are returned to the application itself.

Here's a demo counting round-trips.

In [ ]:
def get_session_id(connection):
    sid, = connection.cursor().execute("SELECT sys_context('userenv', 'sid') FROM dual").fetchone()
    return sid

def get_round_trips(systemconnection, sid):
    sql = """SELECT ss.value
             FROM v$sesstat ss, v$statname sn
             WHERE ss.sid = :sid
             AND ss.statistic# = sn.statistic#
             AND sn.name LIKE '%roundtrip%client%'"""  
    roundtrips, = systemconnection.cursor().execute(sql, [sid]).fetchone()
    return roundtrips

sid = get_session_id(connection)

def do_query(connection, numrows, prefetchrows, arraysize):

    roundtrips = get_round_trips(systemconnection, sid)

    with connection.cursor() as cursor:
        cursor.prefetchrows = prefetchrows
        cursor.arraysize    = arraysize
        cursor.execute("select * from all_objects where rownum <= :nr", [numrows])
        rows = cursor.fetchall()

    roundtrips = get_round_trips(systemconnection, sid) - roundtrips

    print("Number of rows: {:5}, prefetchrows {:4}  arraysize {:4}  roundtrips {:3} ".format(len(rows), prefetchrows, arraysize, roundtrips))


# do_query(connection, number of rows, prefetchrows, arraysize)

print("Default prefetch & arraysize values:")
do_query(connection,                1,            2,       100)
do_query(connection,              100,            2,       100)
do_query(connection,             1000,            2,       100)
do_query(connection,            10000,            2,       100)

print("\n'Unknown' large number of rows:")
do_query(connection,            10000,            2,      1000)
do_query(connection,            10000,         1000,      1000)

print("\n'Page' of rows:")
do_query(connection,               20,           20,        20)
do_query(connection,               20,           21,        20)



With a huge number of rows, tuning `arraysize` is most important.

With a fixed 'page' of rows, set `prefetchrows` to be one larger than the number of rows returned.

# Querying LOB Columns

Documentation reference link: [Using CLOB and BLOB Data](https://cx-oracle.readthedocs.io/en/latest/user_guide/lob_data.html)

When fetching LOBs there are two modes that can be used:
- Fetching as a "locator" for streaming use
- Fetching directly as a string or buffer

### Fetching LOBs for streaming

- this is the default
- it requires more round trips and has more overhead
- it is useful when very long LOBs are being used

In [ ]:
with connection.cursor() as cursor:

    id = 1
    textData = "The quick brown fox jumps over the lazy dog"
    cursor.execute("truncate table TestClobs")
    cursor.execute("insert into TestClobs (IntCol, ClobCol) values (:1, :2)", [id, textData])

    cursor.execute("select ClobCol from TestClobs where IntCol = :1", [id])
    c, = cursor.fetchone()
    print("CLOB length:", c.size())
    print("CLOB data:", c.read())

### Fetching CLOBs as String

- this is much faster than the streaming method
- LOBs are limited to 1 GB (and obviously must all be in memory at once)

This method uses an 'output type handler' to change how query data is fetched from the database.

In [ ]:
with connection.cursor() as cursor:

    def OutputTypeHandler(cursor, name, defaultType, size, precision, scale):
        if defaultType == cx_Oracle.DB_TYPE_CLOB:
            return cursor.var(cx_Oracle.DB_TYPE_LONG, arraysize=cursor.arraysize)

    cursor.outputtypehandler = OutputTypeHandler   # can also set on a connection

    id = 1
    cursor.execute("select ClobCol from TestClobs where IntCol = :1", [id])
    c, = cursor.fetchone()

    print("CLOB length:", len(c))
    print("CLOB data:", c)

The handler can be extended to return BLOBs as Buffers:

```python
def OutputTypeHandler(cursor, name, defaultType, size, precision, scale):
    if defaultType == cx_Oracle.CLOB:
        return cursor.var(cx_Oracle.LONG_STRING, arraysize = cursor.arraysize)
    if defaultType == cx_Oracle.BLOB:
        return cursor.var(cx_Oracle.LONG_BINARY, arraysize = cursor.arraysize)
```

### Sample benchmark
A sample benchmark shows the performance benefit of querying using the direct String method compared with using the default locator stream method. 
![LOB Benchmark](images/lob-benchmark.png)

# Fetching Numbers as Decimals

Oracle's NUMBER format is Decimal, but Python uses floating point, so there can be conversion artifacts.

Documentation reference link:  [Fetched Number Precision](https://cx-oracle.readthedocs.io/en/latest/user_guide/sql_execution.html#fetched-number-precision)

In [ ]:
with connection.cursor() as cursor:
    cursor.execute("select 0.1 from dual")
    v, = cursor.fetchone()
    print('Value =', v, '\tValue * 3 =', v * 3)

Adding an 'output type handler' that converts to Decimal may be useful.

In [ ]:
import decimal

def handler(cursor, name, defaultType, size, precision, scale):
    if defaultType == cx_Oracle.NUMBER:
        return cursor.var(decimal.Decimal, arraysize=cursor.arraysize)

with connection.cursor() as cursor:

    cursor.outputtypehandler = handler   # can also set on a connection

    cursor.execute("select 0.1 from dual")
    v, = cursor.fetchone()
    print('Value =', v, '\tValue * 3 =', v * 3)

## Multiple Values in "WHERE ... IN" Clauses

Documentation reference link: [Binding Multiple Values to a SQL WHERE IN Clause](https://cx-oracle.readthedocs.io/en/latest/user_guide/bind.html#binding-multiple-values-to-a-sql-where-in-clause)

It's important to use bind variables for security.  For small numbers of binds this is simple.  One placeholder must be used for each value in the IN clause.

In [ ]:
sql = """select name
         from SampleQueryTab
         where id in (:id1, :id2, :id3, :id4, :id5)
         order by id"""

with connection.cursor() as cursor:
    cursor.execute(sql,
        id1=5, id2=7, id3=1, id4=3, id5=2)
    for row, in cursor:
        print(row)       

If statements are re-executed but the number of values varies, then pass None (i.e. NULL) for 'missing' values.  This lets the same statement as before be used, which helps performance and scalability.

In [ ]:
with connection.cursor() as cursor:
    cursor.execute(sql,
        id1=2, id2=4, id3=1, id4=None, id5=None)
    for row, in cursor:
        print(row) 

If the number of bind values is not known, and the statement is never rexecuted, you can consider dynamically creating a statement.

In [ ]:
bind_values = [5, 7, 1, 3, 2, 6]
bind_names = [":" + str(i + 1) for i in range(len(bind_values))]
sql = "select name from SampleQueryTab " + \
        "where id in (%s)" % (",".join(bind_names))
print(sql)
with connection.cursor() as cursor:
    cursor.execute(sql, bind_values)
    for row, in cursor:
        print(row)

### One Solution for a huge number of values

This solution uses an Oracle type.  It is great when you have a huge number of bind variables.  If you use it in a more complex statement, check the optimizer plan is still OK.

Accessing the type information takes a few round-trips so the earlier method is better for small numbers of values.

In [ ]:
with connection.cursor() as cursor:
    cursor.execute("CREATE OR REPLACE TYPE number_array AS TABLE OF INTEGER")

In [ ]:
type_obj = connection.gettype("NUMBER_ARRAY")

with connection.cursor() as cursor:
    obj = type_obj.newobject()
    obj.extend([3, 4, 7])
    cursor.execute("""select name
                      from SampleQueryTab
                      where id in (select * from table(:1))""",
                [obj])
    for row, in cursor:
        print(row)

Instead of creating your own type, you can consider using the presupplied types `SYS.ODCINUMBERLIST`, `SYS.ODCIVARCHAR2LIST`, or `SYS.ODCIDATELIST` for NUMBER, VARCHAR2 and DATE, respectively.